In [35]:
import time
import random
import pandas as pd
import json

from tqdm import tqdm

from googletrans import Translator

In [6]:
def google_translate(texts, dest="vi", batch_size=16, delay_per_batch=5):
    translator = Translator()
    is_str = False
    if isinstance(texts, str):
        texts = [texts]
        is_str = True

    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        tries = 5
        while tries > 0:
            try:
                results.extend(translator.translate(batch, dest=dest))
                time.sleep(random.randint(0, 3))
                break
            except Exception as e:
                tries -= 1
                print(e)
                print("Retrying...")
                time.sleep(random.randint(0, 3))

    results = [result.text for result in results]
    if is_str:
        return results[0]
    else:
        return results

In [7]:
google_translate("Anh ta là một người đàn ông tệ", dest="en")

'He is a bad man'

## Config

In [22]:
TRAIN_FILE_PATH = "../datasets/train/math_train.json"
TEST_FILE_PATH = "../datasets/public_test/math_test.json"

TRANSLATED_TRAIN_FILE_PATH = "../datasets/train/math_train_translated.json"
TRANSLATED_TEST_FILE_PATH = "../datasets/public_test/math_test_translated.json"

## Load datasets

In [33]:
with open(TRAIN_FILE_PATH, "r") as f:
    train_json = json.load(f)
    num_samples, train_data = train_json["__count__"], train_json["data"]
    
train_df = pd.DataFrame(train_data)
train_df.head()

,id,question,choices,explanation,answer
0,1,"Một người bán hàng bỏ ra 80,000 đồng tiền vốn ...","[A. 80,000 : 6, B. 80,000 x 6, C. 80,000 : (6 ...","Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồn...","D. (80,000 x 6) : 100"
1,10,8 dm2 24 cm2 = ……… dm2. Số thích hợp điền v...,"[A. 824, B. 82,4, C. 8,24, D. 0,824]","Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = ...","C. 8,24"
2,100,"10% của 11,5m2 là:","[A. 10,15dm2, B. 1,5m2, C. 15,5m2, D. 1,15m2]","10% của 11,5m2 là: 11,5 ${\times}$ 10 : 100 = ...","D. 1,15m2"
3,1004,"Một tam giác có đáy bằng 15 cm và gấp 2,5 l...","[A. 90 cm2, B. 37,5 cm2, C. 45 cm2, D. 18,75 cm2]","Chiều cao của tam giác đó là: 15 : 2,5 = 6 (c...",C. 45 cm2
4,1005,Một tấm thiệp hình chữ nhật có chiều rộ...,"[A. 162 cm2, B. 81 cm2, C. 99 cm2, D. 63 cm2]",Chiều dài hình chữ nhật là: 9 ${\times}$ 2 ...,A. 162 cm2


## Preprocess

In [34]:
train_df['label'] = ""

for i, row in train_df.iterrows():
    question = row["question"]
    choices = row["choices"]
    answer = row["answer"]
    explanation = row.get("explanation", None)
    
    answer_id = choices.index(answer)
    train_df.at[i, "label"] = answer_id
train_df.head()

,id,question,choices,explanation,answer,label
0,1,"Một người bán hàng bỏ ra 80,000 đồng tiền vốn ...","[A. 80,000 : 6, B. 80,000 x 6, C. 80,000 : (6 ...","Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồn...","D. (80,000 x 6) : 100",3
1,10,8 dm2 24 cm2 = ……… dm2. Số thích hợp điền v...,"[A. 824, B. 82,4, C. 8,24, D. 0,824]","Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = ...","C. 8,24",2
2,100,"10% của 11,5m2 là:","[A. 10,15dm2, B. 1,5m2, C. 15,5m2, D. 1,15m2]","10% của 11,5m2 là: 11,5 ${\times}$ 10 : 100 = ...","D. 1,15m2",3
3,1004,"Một tam giác có đáy bằng 15 cm và gấp 2,5 l...","[A. 90 cm2, B. 37,5 cm2, C. 45 cm2, D. 18,75 cm2]","Chiều cao của tam giác đó là: 15 : 2,5 = 6 (c...",C. 45 cm2,2
4,1005,Một tấm thiệp hình chữ nhật có chiều rộ...,"[A. 162 cm2, B. 81 cm2, C. 99 cm2, D. 63 cm2]",Chiều dài hình chữ nhật là: 9 ${\times}$ 2 ...,A. 162 cm2,0


In [28]:
# count all nan in explanation column
train_df[train_df["explanation"].isna()]

,id,question,choices,explanation,answer,label
5,101,10% của 5dm là:,"[A. 0,5 m, B. 5 dm, C. 5 cm, D. 50 cm]",NaN,C. 5 cm,2
7,102,10% của 8 dm là,"[A. 10 cm, B. 70 cm, C. 8 cm, D. 0,8 cm]",NaN,C. 8 cm,2
11,103,12m2 35dm2 = …… m2. Số thích hợp để điền v...,"[A. 1235, B. 12,35, C. 123,5, D. 1,235]",NaN,"B. 12,35",1
16,104,12m7dm = … dm. Số thích hợp điền vào chỗ chấm là:,"[A. 1 207 dm, B. 127 dm, C. 1 270 dm, D. 1 027...",NaN,B. 127 dm,1
30,107,15% của 40 là:,"[A. 6, B. 300, C. 275, D. 60]",NaN,A. 6,0
...,...,...,...,...,...,...
1183,96,1 kg bằng:,"[A. 1000 g, B. 100 g, C. 10 g]",NaN,A. 1000 g,0
1185,97,1 km = … m. Số thích hợp để điền vào chỗ ...,"[A. 1 000, B. 100, C. 10, D. 1]",NaN,A. 1 000,0
1188,98,1/2 của một giờ bằng bao nhiêu phút?,"[A. 12 phút, B. 30 phút, C. 10 phút, D. 20 phút]",NaN,B. 30 phút,1
1193,99,10 là tích của hai số nào?,"[A. 5 và 5, B. 2 và 4, C. 5 và 2, D. 2 và 7]",NaN,C. 5 và 2,2


## Translate datasets

In [36]:
translated_train_data = []
for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    train_id = row["id"]
    question = row["question"]
    choices = row["choices"]
    label = row["label"]
    explanation = row.get("explanation", None)
    
    data2translate = [question, *choices]
    if explanation: data2translate.append(explanation)
    translated_data = google_translate(data2translate, dest="en")
    
    question = translated_data[0]
    choices = translated_data[1:len(choices) + 1]
    if explanation: explanation = translated_data[-1]
    
    translated_train_data.append({
        "id": train_id,
        "question": question,
        "choices": choices,
        "label": label,
        "answer": choices[label],
        "explanation": explanation
    })

translated_train_df = pd.DataFrame(translated_train_data)
translated_train_df.head()

  5%|▍         | 56/1200 [03:18<1:14:54,  3.93s/it]

The read operation timed out
Retrying...


 58%|█████▊    | 698/1200 [39:48<33:47,  4.04s/it]  

The read operation timed out
Retrying...


 79%|███████▉  | 948/1200 [53:34<12:18,  2.93s/it]

The read operation timed out
Retrying...


100%|██████████| 1200/1200 [1:07:30<00:00,  3.38s/it]


,id,question,choices,label,answer,explanation
0,1,"A seller spent 80,000 VND in capital and suffe...","[A. 80,000 : 6, B. 80,000 x 6, C. 80,000 : (6 ...",3,"D. (80,000 x 6) : 100","According to the problem, the loss is equal to..."
1,10,8 dm2 24 cm2 = ……… dm2. The appropriate number...,"[A. 824, B. 82,4, C. 8,24, D. 0,824]",2,"C. 8,24",We have 24 cm2 = 0.24 dm2\n So 8 dm2 24 cm2 = ...
2,100,10% of 11.5m2 is:,"[A. 10,15dm2, B. 1,5m2, C. 15,5m2, D. 1,15m2]",3,"D. 1,15m2",10% of 11.5m2 is: 11.5 ${\times}$ 10 : 100 = 1...
3,1004,A triangle has a base equal to 15 cm and 2.5 t...,"[A. 90 cm2, B. 37,5 cm2, C. 45 cm2, D. 18,75 cm2]",2,C. 45 cm2,The height of that triangle is: 15 : 2.5 = 6 (...
4,1005,A rectangular card has a width of 9 cm and a l...,"[A. 162 cm2, B. 81 cm2, C. 99 cm2, D. 63 cm2]",0,A. 162 cm2,The length of the rectangle is: 9 ${\times}$ 2...


In [37]:
with open(TRANSLATED_TRAIN_FILE_PATH, "w", encoding="utf-8") as f:
    json.dump(translated_train_data, f, indent=4, ensure_ascii=False)